In [185]:
# https://github.com/Protean-Labs/subgrounds
from subgrounds.subgrounds import Subgrounds
from subgrounds.subgraph import SyntheticField
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import date, datetime, timedelta
from operator import add
import warnings
warnings.filterwarnings('ignore')

In [186]:
sg = Subgrounds()
snapshot = sg.load_api('https://hub.snapshot.org/graphql')

In [187]:
proposal_id = '0xe52a4af6307bea40a34f2eaf9a93f11c8abf382020cca2d370ffaa0337620b6b'

snapshot.Vote.date_vote = SyntheticField(
    f = lambda timestamp: str(date.fromtimestamp(timestamp)),
    type_ = SyntheticField.STRING,
    deps = snapshot.Vote.created
)

snapshot.Vote.voter_choice = SyntheticField(
    f = lambda x: 'Allow Ribbon Finance' if x == 1 else 'Do not allow Ribbon Finance',
    type_ = SyntheticField.STRING,
    deps = snapshot.Vote.choice
)

proposal = snapshot.Query.proposal(
    id = proposal_id
)

votes = snapshot.Query.votes(
    first = 226,
    orderBy = 'created',
    orderDirection = 'asc',
    where = {
        'proposal': proposal_id
    }
)

In [188]:
proposal_scores_strat = sg.query_json([
    proposal.scores_by_strategy
    ])
# print(proposal_scores_strat)
scores_by_strategy = list(map(
    add, 
    proposal_scores_strat[0]['x00695cf9dbf02ec0']['scores_by_strategy'][0], 
    proposal_scores_strat[0]['x00695cf9dbf02ec0']['scores_by_strategy'][1]
    )
)
strat_names = ['sOHM v1', 'OHM-DAI SLP', 'sOHM v2', 'sOHM v3', 'sOHM Multichain LP', 'gOHM']

proposal_scores_by_strat = pd.DataFrame(
    data= {'votes': scores_by_strategy, 'asset': strat_names}, 
    columns= ['asset','votes'])
proposal_scores_by_strat

,asset,votes
0,sOHM v1,9.100000e-08
1,OHM-DAI SLP,2.000000e-18
2,sOHM v2,1.246084e+01
3,sOHM v3,6.582513e+04
4,sOHM Multichain LP,1.168486e+04
5,gOHM,3.868778e+05


In [189]:
fig = px.pie(proposal_scores_by_strat, 
    values = 'votes', 
    names = 'asset',
    title = 'Voting Power Used by Asset'
    )
fig.show()

In [190]:
proposal_votes = sg.query_df([
    votes.voter,
    votes.date_vote,
    votes.voter_choice,
    votes.vp
])
# print(proposal_votes)
vp_by_strategy_json = sg.query_json([votes.vp_by_strategy])[0]['x32eae029ffc2c5f8']
# print(vp_by_strategy_json)
vp_by_strategy = {
    'sOHM v1': [], 
    'OHM-DAI SLP': [], 
    'sOHM v2': [], 
    'sOHM v3': [], 
    'sOHM Multichain LP': [], 
    'gOHM': []
}

for i in range(0, len(vp_by_strategy_json)):
    vp_by_strategy['sOHM v1'].append(vp_by_strategy_json[i]['vp_by_strategy'][0]),
    vp_by_strategy['OHM-DAI SLP'].append(vp_by_strategy_json[i]['vp_by_strategy'][1]),
    vp_by_strategy['sOHM v2'].append(vp_by_strategy_json[i]['vp_by_strategy'][2]),
    vp_by_strategy['sOHM v3'].append(vp_by_strategy_json[i]['vp_by_strategy'][3]),
    vp_by_strategy['sOHM Multichain LP'].append(vp_by_strategy_json[i]['vp_by_strategy'][4]),
    vp_by_strategy['gOHM'].append(vp_by_strategy_json[i]['vp_by_strategy'][5])

for key in vp_by_strategy.keys():
    proposal_votes[key] = vp_by_strategy[key]

proposal_votes.head()

,votes_voter,votes_date_vote,votes_voter_choice,votes_vp,sOHM v1,OHM-DAI SLP,sOHM v2,sOHM v3,sOHM Multichain LP,gOHM
0,0xd0bB63e2c183DB414977Eaa6B547601e46E12ed4,2022-04-19,Allow Ribbon Finance,4.310303,0.0,0.0,0.0,4.310303,0.0,1.144218e-09
1,0x8e7742209bf49b7bf2A7007Faa669F8A4cfbbF25,2022-04-19,Allow Ribbon Finance,195.214866,0.0,0.0,0.0,0.000000,0.0,1.952149e+02
2,0xa100cD30f2384E53d149729Db33a84597c296c3d,2022-04-19,Allow Ribbon Finance,264.313736,0.0,0.0,0.0,0.000000,0.0,2.643137e+02
3,0xca6aA0576c2ADc79d2c20e6D53c07f590f62932A,2022-04-19,Do not allow Ribbon Finance,2925.506853,0.0,0.0,0.0,0.000000,0.0,2.925507e+03
4,0x16ff8Ad104Fa6195260A6d4A3F311c5f120dF77d,2022-04-19,Do not allow Ribbon Finance,1143.197250,0.0,0.0,0.0,0.000000,0.0,1.143197e+03


In [191]:
votes_date = proposal_votes.groupby(['votes_date_vote', 'votes_voter_choice']).votes_vp.sum().reset_index()
# votes_date
fig = px.bar(votes_date, 
    x='votes_date_vote', 
    y='votes_vp', 
    color='votes_voter_choice',
    title = 'Daily Votes by Choice',
    labels = dict(votes_date_vote = 'DATE',
        votes_vp = 'Votes')
    )
fig.update_layout(legend_title_text='CHOICE')
fig.show()

In [192]:
fig = px.pie(proposal_votes,
    values = 'votes_vp',
    names = 'votes_voter_choice',
    title = 'Outcome of Proposal TAP-11: Ribbon Finance Whitelist',
    color_discrete_map = {
        'Do not allow Ribbon Finance': 'red',
        'Allow Ribbon Finance': 'blue'
    }
    )

fig.show()

In [193]:
fig = px.pie(proposal_votes,
    values = 'votes_voter',
    names = 'votes_voter_choice',
    title = 'Voters Option',
    color_discrete_map = {
        'Do not allow Ribbon Finance': 'red',
        'Allow Ribbon Finance': 'blue'
    }
    )

fig.show()

In [194]:
fig = px.histogram(proposal_votes,
    x = 'votes_vp',
    # log_y = True,
    marginal = 'box',
    color = 'votes_voter_choice',
    title = 'Distribution of Voting Power by Choice',
    # histnorm = 'count',
    labels = dict(votes_vp = 'VOTING POWER')
    )

fig.update_layout(legend_title_text='CHOICE')
fig.show()